<a href="https://colab.research.google.com/github/yongminkim0501/chatbot_py/blob/main/dart_financial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Discord_bot

/content/drive/MyDrive/Discord_bot


In [18]:
import requests

with open('dart_token.txt', 'r') as file:
  token = file.read().strip()

url = 	"https://opendart.fss.or.kr/api/list.json"
api_key = token
params = {
    'crtfc_key' : api_key
}

results = requests.get(url, params = params).json()

In [ ]:
results_all = pd.DataFrame()  # 전체결과 DataFrame
kosdaq_count = 0  # 코스닥 기업 카운터

while True:
    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': token,
        'page_no': str(page_no),
        'page_count': str(page_count),
    }

    results = requests.get(url, params=params).json()
    results_df = pd.DataFrame(results['list'])

    # K인 경우만 필터링해서 추가
    kosdaq_df = results_df[results_df['corp_cls'] == 'K']
    results_all = pd.concat([results_all, kosdaq_df])

    # 현재까지 수집된 코스닥 기업 수 업데이트
    kosdaq_count = len(results_all)

    # 100개 이상 수집됐으면 중단
    if kosdaq_count >= 100:
        # 정확히 100개만 남기기
        results_all = results_all.head(100)
        break

    # total_page = results['total_page']
    if page_no == 300:
        print(page_no, ',', 300)
        break

    page_no = page_no + 1

print(f"수집된 코스닥 기업 수: {len(results_all)}")
display(results_all)
'''
list 내 내용 정리
cort_cls = 법인 구분
 -> Y (유가), K (코스닥), N (코넥스), E(기타)
corp_name = 종목 명 (법인 명)
corp_code = 고유번호 (공시대상회사의 고유번호)
stock_code = 종목코드 (상장회사의 종목코드)
report_nm = 보고서명 (공시 구분 + 보고서명 + 기타 정보)
  [기재정정] : 본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출
  [첨부정정] : 본 보고서명으로 이미 제출된 보고서의 첨부내용이 변경되어 제출
  [첨부추가] : 본 보고서명으로 이미 제출된 보고서의 첨부서류가 추가되어 제출
  [변경등록] : 본 보고서명으로 이미 제출된 보고서의 유동화 계획이 변경되어 제출
  [연장결정] : 본 보고서명으로 이미 제출된 보고서의 신탁 계약이 연장되어 제출
  [발행조건 확정] : 본 보고서명으로 이미 제출된 보고서의 유가 증권 발행조건이 확정되어 제출
  [정정명령부과] : 본 보고서에 대해서 금감원이 정정명령을 부과한 것
  [정정제출요구] : 본 보고서에 대하여 금감원이 정정제출요구를 부과한 것
rcept_no : 접수번호
fir_nm : 공시 제출인명
rcept_dt : 접수일자
rm : 비고
  유 : 본 공시사항은 한국거래소 유가증권시장본부 소관
  코 : 본 공시사항은 한국거래소 코스닥 시장 본부 소관
  채 : 본 문서는 한국거래소 채권상장법인 공시사항
  넥 : 본 문선느 한국거래소 코넥스 시장 소관
  공 : 본 공시사항은 공정거래위원회 소관
  연 : 본 보고서는 연결부분을 포함한 것
  정 : 본 보고서 제출 후 정정 신고가 있으니 관련 보고서를 참조하시기 바람
  철 : 본 보고서는 철회되었으니 관련 철회 신고서를 참고하시기 바람
'''

In [46]:
import xml.etree.ElementTree as ET
import json

def read_financial_data(filename):
  tree = ET.parse(filename)
  root = tree.getroot()

  financial_data = []

  for document in root.findall('.//document_content'):
    lines = document.text.strip().split("\n")
    for line in lines:
      if line.strip():
        fields = line.split('\t')
        if len(fields) >= 5:
          data = {
              '재무제표종류' : fields[0].strip(),
              '종목코드' : fields[1].strip('[]'),
              '회사명' : fields[2],
              '시장구분' : fields[3],
              '항목명' : fields[11] if len(fields) > 11 else "",
              "당기금액" : fields[12] if len(fields) > 12 else ""
          }
          financial_data.append(data)


  with open('financial_data.json', 'w', encoding = 'utf-8') as f:
    json.dump(financial_data , f, ensure_ascii=False, indent = 4)

  return financial_data


In [42]:
financial_json = []
import os
import traceback
folder = '/content/drive/MyDrive/Discord_bot/financial_Data'
for filename in os.listdir(folder):
  if filename.endswith('.txt'):
    file_path = os.path.join(folder, filename)
    # with open(file_path, 'r', encoding = 'cp949') as f:
    #   content = f.read()
    #   financial_json.append({
    #       'filename' : filename,
    #       'content' : content
    #   })
    try :
      df = pd.read_csv(file_path, delimiter='\t', encoding='cp949')
      financial_json.append({
          'filename' : filename,
          'content':df
      })
    except Exception as e:
      print(f"{filename} 파일 읽기 실패: {str(e)}")
      print(traceback.format_exc())

# for file in financial_json:
#     print(f"\n파일명: {file['filename']}")
#     print(f"데이터 크기: {file['content'].shape}")
#     print("\n처음 5줄:")
#     print(file['content'].head())

print(financial_json)

[{'filename': '재무상태표_7.txt', 'content':               재무제표종류      종목코드     회사명  시장구분   업종        업종명  결산월       결산기준일  \
0     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
1     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
2     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
3     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
4     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
..               ...       ...     ...   ...  ...        ...  ...         ...   
494   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
495   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
496   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
497   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
498   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   

In [50]:
# import pandas as pd

# # DataFrame으로 변환해서 보기
# df = pd.DataFrame(financial_json)
# print(df.head())  # 처음 5개 행 보기
# 1. 전체 데이터를 보기 좋게 출력
print("\n첫 번째 데이터의 구조:")
for key, value in financial_json[0].items():
    print(f"{key}: {value}")


첫 번째 데이터의 구조:
filename: 재무상태표_7.txt
content:               재무제표종류      종목코드     회사명  시장구분   업종        업종명  결산월       결산기준일  \
0     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
1     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
2     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
3     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
4     재무상태표, 은행 - 별도  [192520]    경남은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
..               ...       ...     ...   ...  ...        ...  ...         ...   
494   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
495   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
496   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
497   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저축기관   12  2024-03-31   
498   재무상태표, 은행 - 별도  [016830]  한국씨티은행  기타법인  641  은행 및 저